**[Machine Learning Micro-Course Home Page](https://www.kaggle.com/learn/machine-learning)**

---


# Introduction
Machine learning competitions are a great way to improve your data science skills and measure your progress. 

In this exercise, you will create and submit predictions for a Kaggle competition. You can then improve your model (e.g. by adding features) to improve and see how you stack up to others taking this micro-course.

The steps in this notebook are:
1. Build a Random Forest model with all of your data (**X** and **y**)
2. Read in the "test" data, which doesn't include values for the target.  Predict home values in the test data with your Random Forest model.
3. Submit those predictions to the competition and see your score.
4. Optionally, come back to see if you can improve your model by adding features or changing your model. Then you can resubmit to see how that stacks up on the competition leaderboard.

## Recap
Here's the code you've written so far. Start by running it again.

In [1]:
# Code you have previously used to load data
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from learntools.core import *
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from catboost import CatBoostRegressor



# Path of the file to read. We changed the directory structure to simplify submitting to a competition
iowa_file_path = '../input/train.csv'

home_data = pd.read_csv(iowa_file_path)

print(type(home_data))

imputed_data = home_data.fillna(home_data.mean())

#imputing
#my_imputer = SimpleImputer()
#imputed_data = my_imputer.fit_transform(home_data)

# Create target object and call it y
y = imputed_data.SalePrice
# Create X
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd',
            'YearRemodAdd', 'OverallQual', 'OverallCond', 'HeatingQC']
best_features_according_to_catboost = ['GrLivArea',
                                       'OverallQual',
                                       'GarageCars',
                                       'TotalBsmtSF',
                                       'BsmtQual',
                                       'Fireplaces',
                                       'ExterQual',
                                       '1stFlrSF',
                                       'YearRemodAdd',
                                       'KitchenQual',
                                       'LotArea',
                                       'GarageArea',
                                       'BsmtFinSF1',
                                       'BsmtFinType1',
                                       'TotRmsAbvGrd',
                                       'MSZoning',
                                       '2ndFlrSF',
                                       'FullBath',
                                       'SaleCondition',
                                       'OverallCond']
X = imputed_data[best_features_according_to_catboost]
print(imputed_data.columns)
#all_features_X = imputed_data.loc[:, imputed_data.columns != [['SalePrice', 'Exterior2nd']]]
all_features_X = imputed_data.drop(['SalePrice', 
                                    'Exterior2nd', 
                                    'RoofMatl',
                                    'Utilities',
                                    'Heating',
                                    'HouseStyle',
                                    'Exterior1st',
                                    'PoolQC',
                                    'RoofMatl',
                                    'Electrical',
                                    'Condition2',
                                    'GarageQual',
                                    'MiscFeature'], axis=1)
print(all_features_X)
#all_features_X = imputed_data - imputed_data.SalePrice
print(all_features_X.columns.values)

#s = pd.Series(list(['Ex', 'Gd', 'TA', 'Fa', 'Po']))
#pd.get_dummies('HeatingQC')
#print(pd.get_dummies(X))
X_dummies = pd.get_dummies(all_features_X)
#one-hot-encoded-X = pd.get_dummies(features)
all_X_dummies = pd.get_dummies(all_features_X)


# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(all_X_dummies, y, random_state=1)

# Make validation predictions and calculate mean absolute error
iowa_model = DecisionTreeRegressor(random_state=1)
iowa_model.fit(train_X, train_y) 
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

# Using best value for max_leaf_nodes
iowa_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
iowa_model.fit(train_X, train_y)
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes: {:,.0f}".format(val_mae))

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y) 
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)
print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

#XGB
xgb_model = XGBRegressor(n_estimators=1000)
xgb_model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(val_X, val_y)], verbose=False)
xgb_val_predictions = xgb_model.predict(val_X)
xgb_val_mae = mean_absolute_error(xgb_val_predictions, val_y)
print("Validation MAE for XGB Model: {:,.0f}".format(xgb_val_mae))

#CatBoost
#catboost_model = CatBoostRegressor()
#catboost_model.fit(train_X, train_y)
#print(catboost_model.get_feature_importance(data=None,
                      # prettified=True,
                      # thread_count=-1,
                      # verbose=False))
#catboost_model_val_predictions = catboost_model.predict(val_X)
#catboost_model_val_mae = mean_absolute_error(catboost_model_val_predictions, val_y)
#print("Validation MAE for Catboost Model: {:,.0f}".format(catboost_model_val_mae))

<class 'pandas.core.frame.DataFrame'>
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQ

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Validation MAE for Random Forest Model: 17,681
Validation MAE for XGB Model: 14,875


# Creating a Model For the Competition

Build a Random Forest model and train it on all of **X** and **y**.  

In [2]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
xgb_model_on_full_data = XGBRegressor(n_estimators=1000)
#print(all_X_dummies.columns)
print(len(all_X_dummies.columns))
# fit rf_model_on_full_data on all data from the training data
xgb_model_on_full_data.fit(all_X_dummies, y, early_stopping_rounds=5, eval_set=[(val_X, val_y)], verbose=False)


209


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# Make Predictions
Read the file of "test" data. And apply your model to make predictions

In [3]:


# path to file you will use for predictions
test_data_path = '../input/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)

#imputing
my_imputer = SimpleImputer()
new_imputed_data  = test_data.fillna(home_data.mean())
#new_imputed_data = new_imputed_data.as_matrix()

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = new_imputed_data.drop(['Exterior2nd', 
                                    'RoofMatl',
                                    'Utilities',
                                    'Heating',
                                    'HouseStyle',
                                    'Exterior1st',
                                    'PoolQC',
                                    'RoofMatl',
                                    'Electrical',
                                    'Condition2',
                                    'GarageQual',
                                    'MiscFeature'], axis=1)
print(test_X.columns.values)
test_X_dummies = pd.get_dummies(test_X)
new_test_X_dummies = test_X_dummies
print(test_X_dummies.columns.values)
#test_X_dummies = pd.get_dummies(all_features_X)
#test_X = imputed_data[best_features_according_to_catboost]
#test_X_dummies = pd.get_dummies(all_features_X)

 # predict is not working without this code


# make predictions which we will submit. 

test_preds = xgb_model_on_full_data.predict(test_X_dummies)

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)

['Id' 'MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'Alley'
 'LotShape' 'LandContour' 'LotConfig' 'LandSlope' 'Neighborhood'
 'Condition1' 'BldgType' 'OverallQual' 'OverallCond' 'YearBuilt'
 'YearRemodAdd' 'RoofStyle' 'MasVnrType' 'MasVnrArea' 'ExterQual'
 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'HeatingQC' 'CentralAir' '1stFlrSF' '2ndFlrSF'
 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath' 'FullBath'
 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual' 'TotRmsAbvGrd'
 'Functional' 'Fireplaces' 'FireplaceQu' 'GarageType' 'GarageYrBlt'
 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageCond' 'PavedDrive'
 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch' '3SsnPorch' 'ScreenPorch'
 'PoolArea' 'Fence' 'MiscVal' 'MoSold' 'YrSold' 'SaleType' 'SaleCondition']
['Id' 'MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'Bsmt

# Test Your Work
After filling in the code above:
1. Click the **Commit and Run** button. 
2. After your code has finished running, click the small double brackets **<<** in the upper left of your screen.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
3. Go to the output tab at top of your screen. Select the button to submit your file to the competition.  
4. If you want to keep working to improve your model, select the edit button. Then you can change your model and repeat the process.

Congratulations, you've started competing in Machine Learning competitions.

# Continuing Your Progress
There are many ways to improve your model, and **experimenting is a great way to learn at this point.**

The best way to improve your model is to add features.  Look at the list of columns and think about what might affect home prices.  Some features will cause errors because of issues like missing values or non-numeric data types. 

Level 2 of this micro-course will teach you how to handle these types of features. You will also learn to use **xgboost**, a technique giving even better accuracy than Random Forest.


# Other Micro-Courses
The **[Pandas Micro-Course](https://kaggle.com/Learn/Pandas)** will give you the data manipulation skills to quickly go from conceptual idea to implementation in your data science projects. 

You are also ready for the **[Deep Learning](https://kaggle.com/Learn/Deep-Learning)** micro-course, where you will build models with better-than-human level performance at computer vision tasks.

---
**[Machine Learning Micro-Course Home Page](https://www.kaggle.com/learn/machine-learning)**

